1) Для головоломок типа "Кубик" мы знаем правила 🎲 __mi.mi.mi = -mi__ и __mi.mi.mi.mi = пусто__. Давайте расширим их для всех трёх видов головоломок, создав правила изменения знака в форме __mi\*\*(n) = -mi\*\*(n)__:

   - Для "Кубика", __n=2__, или __mi.mi = -mi.-mi__. Используя правило отмены пар (https://www.kaggle.com/code/cl12102783/cancel-pairs-for-all-puzzles), получаем:
     - mi.mi.mi = -mi.-mi.mi = -mi
     - mi.mi.mi.mi = -mi.-mi.mi.mi = пусто
     - Также существует коммутативное правило для "Кубика": mi.m?.mi.m?.mi = неупорядоченный(-mi.m?.m?)
   
   - Для "Венка", n зависит от его размера:
     - Если размер венка чётный, __n = размер/2__. Например, wreath_6/6 имеет n=3
     - Если размер венка нечётный, __n = размер__. Например, wreath_7/7 имеет n=7
     - "Венок" некоммутативен, поэтому замена производится без изменения местоположения, аналогично https://www.kaggle.com/code/cl12102783/cancel-pairs-for-all-puzzles
   
   - Для "Глобуса":
     - fi некоммутативно с fi=-fi (https://www.kaggle.com/code/cl12102783/cancel-pairs-for-all-puzzles), или n=1
     - ri коммутативно, n равно конечному числу типа. Например, globe_1/8 имеет n=8

2) Кроме того, для применения правила изменения знаков, когда количество mi превышает n, происходит уменьшение. Правило уменьшения: __n_left = n-(count - n)__:
   - Если n_left>0, то будет __-mi\*\*(n_left)__. Например, для "Кубика" mi.mi.mi имеет n=2, так что 2-(2-1)=1, становится -mi 🔄
   - Если n_left<0, то будет __mi\*\*(n_left)__. Например, для globe_1/8, ri\*\*(18) имеет n=8, так что 8-(18-8)=-2, становится ri\*\*(2) 🔀
   - Если n_left=0, то все mi отменяются в одной группе ходов. Например, для "Кубика" mi.mi.mi.mi имеет n=2, так что 2-(4-2)=0, отменяются 🚫


In [27]:
# Импорт необходимых библиотек
import pandas as pd
from collections import deque
import tqdm
import glob
import numpy as np
from collections import Counter

# Задание пути к файлам данных
path = '/kaggle/input/santa-2023/'

# Загрузка данных из CSV файлов в pandas DataFrame
df_puzzles = pd.read_csv(path + 'puzzles.csv')
df_puzzle_info = pd.read_csv(path + 'puzzle_info.csv')

# Выбор всех файлов с высокими результатами
files = [i for i in glob.glob('/kaggle/input/*/*.csv') if 'submission' in i and '/santa-2023/' not in i]


In [28]:
# Функция для отмены группы ходов для заданной головоломки
def cancel_group(moves, group):
    def get_grp(elem):
        return elem[1] if elem.startswith('-') else elem[0]

    def drop_grp(elem, group):
        # Извлечение основного типа головоломки (cube, globe)
        grp = group.split('_')[0]
        
        # Проверка на специфичность головоломки "globe" и хода "-f"
        if grp == 'globe' and elem[-1].replace('-', '')[0] == 'f':
            return elem
            
        # Конфигурация размера группы для различных типов головоломок
        config = dict(
            cube = 2,
            globe = int(group.split('/')[-1]),
        )
        grp_size = config[grp]
        move = elem.copy()
        flag = True
        while flag:
            flag = False
            cnt = Counter(move)
            for k, v in cnt.items():
                if v > grp_size:
                    flag = True
                    sign_opp = '' if k.startswith('-') else '-'
                    num_add = grp_size - (v - grp_size)
                    for _ in range(abs(num_add)):
                        if num_add > 0:
                            move.append(sign_opp + k.replace('-', ''))
                        elif num_add < 0:
                            move.append(k)
                    for _ in range(v):
                        move.remove(k)
                    break
        return move
    
    def solver(elem, grp):
        move = elem.copy()
        grp_name = grp.split('_')[0]
        # Выполнение сокращения группы, если тип головоломки не "wreath"
        if grp_name != 'wreath':
            move = drop_grp(move, grp)
        return move
    
    # Добавление точки для удобства обработки последнего хода
    moves = moves + '.'
    win = deque()
    result = deque()
    move = ''
    
    for i in moves:
        if i != '.':
            move += i
        else:
            if len(win) < 1:
                win.append(move)
                move = ''
                continue
            grp_last, grp_new = get_grp(win[-1]), get_grp(move)
            if grp_last == grp_new:
                win.append(move)
            else:
                result.extend(solver(win, group))
                win = deque([move])
            move = ''
     
    # Сбор оставшейся части
    if len(win) > 0:
        result.extend(solver(win, group))
    return '.'.join(result)


# Функция сокращения ходов для головоломки типа "wreath"
def drop_grp_wreath(moves, group):
    # Извлечение числа из типа головоломки "wreath"
    kind = group.split('/')[-1]
    kind = int(int(kind) / 2) if int(kind) % 2 == 0 else int(kind)
    
    # Добавление точки для удобства обработки последнего хода
    moves += '.'
    result = deque()
    win = deque()
    move = ''
    
    for i in moves:
        if i !='.':
            move += i
        else:
            if not win:
                win.append(move)
                move = ''
                continue
            if move != win[-1]:
                if len(win) > kind:
                    # Печать для отладки, может быть удалена
                    print('I am here')
                    times = kind - (len(win) - kind)
                    if times > 0:
                        case_opp = win[-1][1] if len(win[-1]) > 1 else '-' + win[-1]
                        result.extend([case_opp] * times)
                    elif times < 0:
                        result.extend([win[-1]] * abs(times))
                else:
                    result.extend(win)
                win = deque([move]) 
            else:
                win.append(move)
            move = ''
    
    # Сбор оставшейся части
    if win:
        result.extend(win)
    return '.'.join(result)


# Функция многократных попыток улучшения хода в зависимости от типа головоломки
def multiple_try(elem, group):
    # Инициализация начальных значений
    len_old = len(elem.split('.'))
    move_old = elem
    flag = True
    
    # Пока есть возможность улучшения хода
    while flag:
        if group.split('_')[0] == 'wreath':
            move_new = drop_grp_wreath(move_old, group)
        else:
            move_new = cancel_group(move_old, group)
            
        len_new = len(move_new.split('.'))
        # Если новый ход короче, обновляем значения
        if len_new < len_old:
            move_old = move_new
            len_old = len_new
        else:
            # Завершаем цикл, если ход не улучшился
            flag = False
    return move_old


In [29]:
# # Функция для отмены группы ходов для заданной головоломки
# def cancel_group(moves, group):
#     def get_grp(elem):
#         return elem[1] if elem.startswith('-') else elem[0]

#     def drop_grp(elem, group):
#         # Извлечение основного типа головоломки (cube, globe)
#         grp = group.split('_')[0]
        
#         # Проверка на специфичность головоломки "globe" и хода "-f"
#         if grp == 'globe' and elem[-1].replace('-', '')[0] == 'f':
#             return elem
            
#         # Конфигурация размера группы для различных типов головоломок
#         config = dict(
#             cube = 2,
#             globe = int(group.split('/')[-1]),
#         )
#         grp_size = config[grp]
#         move = elem.copy()
#         flag = True
#         while flag:
#             flag = False
#             cnt = Counter(move)
#             for k, v in cnt.items():
#                 if v > grp_size:
#                     flag = True
#                     sign_opp = '' if k.startswith('-') else '-'
#                     num_add = grp_size - (v - grp_size)
#                     for _ in range(abs(num_add)):
#                         if num_add > 0:
#                             move.append(sign_opp + k.replace('-', ''))
#                         elif num_add < 0:
#                             move.append(k)
#                     for _ in range(v):
#                         move.remove(k)
#                     break
#         return move
    
#     def solver(elem, grp):
#         move = elem.copy()
#         grp_name = grp.split('_')[0]
#         # Выполнение сокращения группы, если тип головоломки не "wreath"
#         if grp_name != 'wreath':
#             move = drop_grp(move, grp)
#         return move
    
#     # Добавление точки для удобства обработки последнего хода
#     moves = moves + '.'
#     win = deque()
#     result = deque()
#     move = ''
    
#     for i in moves:
#         if i != '.':
#             move += i
#         else:
#             if len(win) < 1:
#                 win.append(move)
#                 move = ''
#                 continue
#             grp_last, grp_new = get_grp(win[-1]), get_grp(move)
#             if grp_last == grp_new:
#                 win.append(move)
#             else:
#                 result.extend(solver(win, group))
#                 win = deque([move])
#             move = ''
     
#     # Сбор оставшейся части
#     if len(win) > 0:
#         result.extend(solver(win, group))
#     return '.'.join(result)


# # Функция сокращения ходов для головоломки типа "wreath"
# def drop_grp_wreath(moves, group):
#     # Извлечение числа из типа головоломки "wreath"
#     kind = group.split('/')[-1]
#     kind = int(int(kind) / 2) if int(kind) % 2 == 0 else int(kind)
    
#     # Добавление точки для удобства обработки последнего хода
#     moves += '.'
#     result = deque()
#     win = deque()
#     move = ''
    
#     for i in moves:
#         if i !='.':
#             move += i
#         else:
#             if not win:
#                 win.append(move)
#                 move = ''
#                 continue
#             if move != win[-1]:
#                 if len(win) > kind:
#                     # Печать для отладки, может быть удалена
#                     print('I am here')
#                     times = kind - (len(win) - kind)
#                     if times > 0:
#                         case_opp = win[-1][1] if len(win[-1]) > 1 else '-' + win[-1]
#                         result.extend([case_opp] * times)
#                     elif times < 0:
#                         result.extend([win[-1]] * abs(times))
#                 else:
#                     result.extend(win)
#                 win = deque([move]) 
#             else:
#                 win.append(move)
#             move = ''
    
#     # Сбор оставшейся части
#     if win:
#         result.extend(win)
#     return '.'.join(result)


# # Функция многократных попыток улучшения хода в зависимости от типа головоломки
# def multiple_try(elem, group):
#     # Инициализация начальных значений
#     len_old = len(elem.split('.'))
#     move_old = elem
#     flag = True
    
#     # Пока есть возможность улучшения хода
#     while flag:
#         if group.split('_')[0] == 'wreath':
#             move_new = drop_grp_wreath(move_old, group)
#         else:
#             move_new = cancel_group(move_old, group)
            
#         len_new = len(move_new.split('.'))
#         # Если новый ход короче, обновляем значения
#         if len_new < len_old:
#             move_old = move_new
#             len_old = len_new
#         else:
#             # Завершаем цикл, если ход не улучшился
#             flag = False
#     return move_old


Этот код выполняет следующие действия:

Создает словарь result для хранения результатов.
Итерирует по файлам, используя tqdm для отслеживания прогресса.
Открывает каждый файл и обрабатывает строки, игнорируя заголовок.
Извлекает идентификатор (id_) и ход (move) из каждой строки.
Определяет тип головоломки (group) для данного идентификатора из DataFrame df_puzzles.
Применяет функцию multiple_try к ходу.
Обновляет результат в словаре result, учитывая наименьшую длину хода.


In [30]:
# Создание словаря для хранения результатов
result = {}

# Итерация по файлам с использованием tqdm для отображения прогресса
for file in tqdm.tqdm(files):
    # Открытие файла для чтения с использованием контекстного менеджера
    with open(file, 'r') as f:
        # Игнорирование заголовка сразу после открытия файла
        next(f, None)

        for row in f:
            # Извлечение данных из строки
            id_, move = row.strip().split(',')
            id_ = int(id_)

            # Определение типа головоломки для данного идентификатора без использования DataFrame
            group = df_puzzles.at[id_, 'puzzle_type']

            # Применение функции multiple_try к ходу
            move = multiple_try(move, group)

            # Обновление результата в словаре
            if id_ not in result:
                result[id_] = move
            else:
                # Сравнение ходов и выбор наименьшего по длине
                if len(move.split('.')) < len(result[id_].split('.')):
                    result[id_] = move


100%|██████████| 3/3 [00:11<00:00,  4.00s/it]


In [31]:
df_sub = pd.DataFrame()
df_sub['id'] = result.keys()
df_sub['moves'] = result.values()
df_sub.to_csv('submission.csv', index=False)

In [32]:
df_sub.moves.str.split('.').apply(lambda x: len(x)).sum()

827041